# Importações


In [10]:
#x = 0
#if x == 1 :
  #!pip install scikit-optimize
  #!pip install catboost
 #!pip install category_encoders

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skopt import BayesSearchCV
from category_encoders import CatBoostEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from xgboost import  XGBClassifier

In [12]:
df_test = pd.read_csv('/test.csv')
df_train = pd.read_csv('/train.csv')


#Análise exploratória

In [13]:
print(df_train.shape,df_test.shape)

(17836, 12) (7644, 11)


In [14]:
df_train.head()

,id_do_caso,continente,educacao_do_empregado,tem_experiencia_de_trabalho,requer_treinamento_de_trabalho,num_de_empregados,ano_de_estabelecimento,regiao_de_emprego,salario_prevalecente,unidade_de_salario,posicao_em_tempo_integral,status_do_caso
0,EZYV10567,Europa,Ensino Médio,N,S,2087,1855,Sul,69711.24,Ano,S,Negado
1,EZYV5505,Ásia,Mestrado,S,N,5991,2003,Meio-Oeste,52931.38,Ano,S,Aprovado
2,EZYV5207,Ásia,Ensino Médio,N,N,1426,2000,Ilha,110830.21,Ano,S,Negado
3,EZYV7545,Ásia,Ensino Médio,N,N,3846,1992,Meio-Oeste,91884.68,Semana,S,Negado
4,EZYV16071,Ásia,Ensino Superior,S,N,3957,1949,Nordeste,138155.24,Ano,S,Aprovado


In [15]:
df_train.describe() #Aqui notamos que tem uma red flag com o salário, está muito discrepante o min e o max.

,num_de_empregados,ano_de_estabelecimento,salario_prevalecente
count,17836.00000,17836.000000,17836.000000
mean,5726.50185,1979.494898,74327.428401
std,23321.37739,42.097194,52795.859573
min,-26.00000,1800.000000,2.136700
25%,1023.00000,1976.000000,33892.912500
50%,2117.00000,1997.000000,70106.920000
75%,3504.25000,2005.000000,107564.715000
max,602069.00000,2016.000000,319210.270000


In [16]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17836 entries, 0 to 17835
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id_do_caso                      17836 non-null  object 
 1   continente                      17836 non-null  object 
 2   educacao_do_empregado           17836 non-null  object 
 3   tem_experiencia_de_trabalho     17836 non-null  object 
 4   requer_treinamento_de_trabalho  17836 non-null  object 
 5   num_de_empregados               17836 non-null  int64  
 6   ano_de_estabelecimento          17836 non-null  int64  
 7   regiao_de_emprego               17836 non-null  object 
 8   salario_prevalecente            17836 non-null  float64
 9   unidade_de_salario              17836 non-null  object 
 10  posicao_em_tempo_integral       17836 non-null  object 
 11  status_do_caso                  17836 non-null  object 
dtypes: float64(1), int64(2), object(

<h2>Descobrindo as variáveis categóricas

In [17]:
lista_var_categoricas = [x for x in df_train.columns if df_train[x].dtypes == 'object']
lista_var_categoricas

['id_do_caso',
 'continente',
 'educacao_do_empregado',
 'tem_experiencia_de_trabalho',
 'requer_treinamento_de_trabalho',
 'regiao_de_emprego',
 'unidade_de_salario',
 'posicao_em_tempo_integral',
 'status_do_caso']

In [18]:
#vendo as variáveis únicas de cada coluna
listas_unicas = [f'{x} : {list(df_train[x].unique())[:10]}' for x in lista_var_categoricas]
listas_unicas

["id_do_caso : ['EZYV10567', 'EZYV5505', 'EZYV5207', 'EZYV7545', 'EZYV16071', 'EZYV3753', 'EZYV18223', 'EZYV16032', 'EZYV19836', 'EZYV4565']",
 "continente : ['Europa', 'Ásia', 'América do Sul', 'América do Norte', 'África', 'Oceania']",
 "educacao_do_empregado : ['Ensino Médio', 'Mestrado', 'Ensino Superior', 'Doutorado']",
 "tem_experiencia_de_trabalho : ['N', 'S']",
 "requer_treinamento_de_trabalho : ['S', 'N']",
 "regiao_de_emprego : ['Sul', 'Meio-Oeste', 'Ilha', 'Nordeste', 'Oeste']",
 "unidade_de_salario : ['Ano', 'Semana', 'Hora', 'Mês']",
 "posicao_em_tempo_integral : ['S', 'N']",
 "status_do_caso : ['Negado', 'Aprovado']"]

In [19]:
#Aqui vemos que a unidade de salário varia
unidades_de_salario = list(df_train['unidade_de_salario'].unique())
unidades_de_salario

['Ano', 'Semana', 'Hora', 'Mês']

# Pré-processamento

<h3> Limpando valores negativos no número de empregados

In [20]:
df_train.loc[df_train["num_de_empregados"] < 0]
df_train.loc[[85,645]]

,id_do_caso,continente,educacao_do_empregado,tem_experiencia_de_trabalho,requer_treinamento_de_trabalho,num_de_empregados,ano_de_estabelecimento,regiao_de_emprego,salario_prevalecente,unidade_de_salario,posicao_em_tempo_integral,status_do_caso
85,EZYV17007,Ásia,Doutorado,S,N,-11,1984,Oeste,25753.51,Ano,S,Negado
645,EZYV17845,Ásia,Ensino Superior,N,N,-14,2012,Oeste,29325.85,Ano,S,Negado


In [21]:
df_train.loc[[85,645]]

,id_do_caso,continente,educacao_do_empregado,tem_experiencia_de_trabalho,requer_treinamento_de_trabalho,num_de_empregados,ano_de_estabelecimento,regiao_de_emprego,salario_prevalecente,unidade_de_salario,posicao_em_tempo_integral,status_do_caso
85,EZYV17007,Ásia,Doutorado,S,N,-11,1984,Oeste,25753.51,Ano,S,Negado
645,EZYV17845,Ásia,Ensino Superior,N,N,-14,2012,Oeste,29325.85,Ano,S,Negado


<h3>Criando variável relativa ao porte da empresa

In [22]:
media_empregados_train = df_train["num_de_empregados"].mean()
media_empregados_test = df_test["num_de_empregados"].mean()

df_train['empresa_grande'] = df_train['num_de_empregados'].apply(lambda x: 1 if x > media_empregados_train else 0)
df_test['empresa_grande'] = df_test['num_de_empregados'].apply(lambda x: 1 if x > media_empregados_test else 0)


In [23]:
df_train['empresa_pequena'] = df_train['num_de_empregados'].apply(lambda x: 1 if x  <= int(media_empregados_train * 0.20) else 0)
df_test['empresa_pequena'] = df_test['num_de_empregados'].apply(lambda x: 1 if x  <= int(media_empregados_test * 0.20) else 0)

In [24]:
df_train[["num_de_empregados", "empresa_grande"]].sample()

,num_de_empregados,empresa_grande
12949,2099,0


<h3> Convertendo os salarios para valor mensal

In [25]:
def altera_unidade_salario(row):

    if row["unidade_de_salario"] == "Ano":
        return row["salario_prevalecente"] / 12

    elif row["unidade_de_salario"] == "Semana":
        return row["salario_prevalecente"] * 4

    elif row["unidade_de_salario"] == "Hora" and row["posicao_em_tempo_integral"] == "S":
        return row["salario_prevalecente"] * 8 * 21

    elif row["unidade_de_salario"] == "Hora" and row["posicao_em_tempo_integral"] == "N":

        return row["salario_prevalecente"] * 6 * 21
    else:
        return row["salario_prevalecente"]




df_train["salario_prevalecente"] = df_train.apply(altera_unidade_salario,  axis = 1)
df_train["salario_prevalecente"] = df_train.apply(altera_unidade_salario,  axis = 1)

df_train.rename(columns={"salario_prevalecente": 'salario_anual'}, inplace=True)
df_test.rename(columns={"salario_prevalecente": 'salario_anual'}, inplace=True)

<h3> Criando variável tempo de mercado

In [26]:
df_train["tempo_de mercado"] = 2023 - df_train["ano_de_estabelecimento"]
df_test["tempo_de mercado"] = 2023 - df_test["ano_de_estabelecimento"]

<h3>Dropando variáveis

In [27]:
df_train.drop(["ano_de_estabelecimento"], axis=1, inplace=True)
df_test.drop(["ano_de_estabelecimento"], axis=1, inplace=True)

In [28]:
df_train.drop(["unidade_de_salario"], axis=1, inplace=True)
df_test.drop(["unidade_de_salario"], axis=1, inplace=True)

In [29]:
df_train

,id_do_caso,continente,educacao_do_empregado,tem_experiencia_de_trabalho,requer_treinamento_de_trabalho,num_de_empregados,regiao_de_emprego,salario_anual,posicao_em_tempo_integral,status_do_caso,empresa_grande,empresa_pequena,tempo_de mercado
0,EZYV10567,Europa,Ensino Médio,N,S,2087,Sul,4.841058e+02,S,Negado,0,0,168
1,EZYV5505,Ásia,Mestrado,S,N,5991,Meio-Oeste,3.675790e+02,S,Aprovado,1,0,20
2,EZYV5207,Ásia,Ensino Médio,N,N,1426,Ilha,7.696542e+02,S,Negado,0,0,23
3,EZYV7545,Ásia,Ensino Médio,N,N,3846,Meio-Oeste,1.470155e+06,S,Negado,0,0,31
4,EZYV16071,Ásia,Ensino Superior,S,N,3957,Nordeste,9.594114e+02,S,Aprovado,0,0,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17831,EZYV17821,Ásia,Mestrado,S,N,95347,Oeste,4.839739e+02,S,Aprovado,1,0,28
17832,EZYV13200,Europa,Ensino Médio,N,N,1483,Sul,9.460946e+02,S,Negado,0,0,137
17833,EZYV8903,América do Norte,Mestrado,N,N,1504,Nordeste,8.207451e+02,S,Negado,0,0,13
17834,EZYV6163,América do Norte,Doutorado,S,N,251967,Meio-Oeste,6.467597e+02,S,Aprovado,1,0,13


<h3>Lidando com variaveis categóricas




In [30]:
lista_ignora = ['id_do_caso', 'continente', 'regiao_de_emprego']
lista_ignora= ['id_do_caso','continente', 'regiao_de_emprego']

lista_var_categoricas_train = [x for x in df_test.columns if df_test[x].dtypes == 'object' and x not in lista_ignora]
lista_var_categoricas_test = [x for x in df_test.columns if df_test[x].dtypes == 'object' and x not in lista_ignora]


In [31]:
lista_var_categoricas_train

['educacao_do_empregado',
 'tem_experiencia_de_trabalho',
 'requer_treinamento_de_trabalho',
 'posicao_em_tempo_integral']

In [32]:

ordinal_encoder= OrdinalEncoder(categories=[
    ['Ensino Médio','Ensino Superior', 'Mestrado', 'Doutorado'],
    ['N', 'S'],
    ['N', 'S'],
    ['N', 'S']]
    )

print()

In [33]:
id_train_guardar = df_train["id_do_caso"]
id_test_guardar = df_test["id_do_caso"]

X_train = df_train.drop(["status_do_caso", "id_do_caso"], axis=1)
y_train = df_train[["status_do_caso"]]

X_test = df_test.drop(["id_do_caso"], axis=1)


In [34]:
X_train["num_com_salario"] = X_train["salario_anual"] / X_train["num_de_empregados"]
X_test["num_com_salario"] = X_test["salario_anual"] / X_test["num_de_empregados"]

In [35]:
lista_cat = ['continente', 'regiao_de_emprego']

In [36]:
y_train = y_train["status_do_caso"].apply(lambda x: 1 if x == 'Aprovado' else 0)

In [37]:
X_train[lista_var_categoricas_train] = ordinal_encoder.fit_transform(X_train[lista_var_categoricas_train])
X_test[lista_var_categoricas_test] = ordinal_encoder.fit_transform(X_test[lista_var_categoricas_test])


In [38]:
encoder = CatBoostEncoder()
encoder.fit(X_train[lista_cat], y_train)

X_train[lista_cat] = encoder.transform(X_train[lista_cat])
X_test[lista_cat] = encoder.transform(X_test[lista_cat])

In [39]:
X_train["Xp_requer"] = ((X_train["tem_experiencia_de_trabalho"] - X_train["requer_treinamento_de_trabalho"]) + 1)
X_test["Xp_requer"] = ((X_test["tem_experiencia_de_trabalho"] - X_test["requer_treinamento_de_trabalho"]) + 1)

In [40]:
X_train["valor_hora_qualifica"] = (X_train["educacao_do_empregado"] + 1) * ((X_train["posicao_em_tempo_integral"] * 2)) + 1
X_test["valor_hora_qualifica"] = (X_test["educacao_do_empregado"] + 1) * ((X_test["posicao_em_tempo_integral"] * 2)) + 1

In [41]:
X_train

,continente,educacao_do_empregado,tem_experiencia_de_trabalho,requer_treinamento_de_trabalho,num_de_empregados,regiao_de_emprego,salario_anual,posicao_em_tempo_integral,empresa_grande,empresa_pequena,tempo_de mercado,num_com_salario,Xp_requer,valor_hora_qualifica
0,0.793849,0.0,0.0,1.0,2087,0.700523,4.841058e+02,1.0,0,0,168,0.231963,0.0,3.0
1,0.653635,2.0,1.0,0.0,5991,0.759850,3.675790e+02,1.0,1,0,20,0.061355,2.0,7.0
2,0.653635,0.0,0.0,0.0,1426,0.604097,7.696542e+02,1.0,0,0,23,0.539729,1.0,3.0
3,0.653635,0.0,0.0,0.0,3846,0.759850,1.470155e+06,1.0,0,0,31,382.255559,1.0,3.0
4,0.653635,1.0,1.0,0.0,3957,0.628168,9.594114e+02,1.0,0,0,74,0.242459,2.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17831,0.653635,2.0,1.0,0.0,95347,0.625418,4.839739e+02,1.0,1,0,28,0.005076,2.0,7.0
17832,0.793849,0.0,0.0,0.0,1483,0.700523,9.460946e+02,1.0,0,0,137,0.637960,1.0,3.0
17833,0.621807,2.0,0.0,0.0,1504,0.628168,8.207451e+02,1.0,0,0,13,0.545708,1.0,7.0
17834,0.621807,3.0,1.0,0.0,251967,0.759850,6.467597e+02,1.0,1,0,13,0.002567,2.0,9.0


<h3> Preparando os dados finais para serem usados no modelo

#Modelagem

<h4> XGBClassifier

# Modelos de validação

In [42]:
classificador_xgb = XGBClassifier(learning_rate=0.01, n_estimators=300, max_depth=4,
                           subsample=1, colsample_bytree=1, scale_pos_weight  = 0.68)

classificador_xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [43]:
def validation(model, X, y):

  #leafs = [x for x in range(50, 1150, 50)]
  best = {'f1-score': 8128509921691552,
          'Roc': 0.7612680532922118}

  f1_cv = cross_val_score(classificador_xgb, X, y, scoring='f1').mean()
  roc_cv = cross_val_score(classificador_xgb, X, y, scoring='roc_auc').mean()

  print(f'F1-score: {f1_cv}')
  print(f'Roc:      {roc_cv}')
  print(roc_cv >= 0.6795677301669668)
  print()

validation(classificador_xgb, X_train, y_train)
#F1:  0.8195849725949416
#Roc: 0.6795677301669668

F1-score: 0.8101508489754308
Roc:      0.7819135847510703
True



In [44]:
predicoes_classifier = classificador_xgb.predict(X_test)

In [45]:
parametros = {
    "n_estimators": [1000, 10000],
    "objective": ["binary:logistic", "binary:hinge"],
    'tree_method':['hist'],
    "eta": [0.1, 0.5],
    "max_depth":[3, 8],
}


In [46]:
y = 0

if y == 1:

  kfold = KFold(n_splits=5, shuffle=True, random_state=42)

  grid = BayesSearchCV(classificador_xgb, search_spaces=parametros, cv=kfold, n_jobs=-1, return_train_score=True)

  grid.fit(X=X_train, y=y_train)

  best_param = grid.best_params_

  print(best_param)

# Preparando submissão

In [47]:
series_predicoes2 = pd.Series(predicoes_classifier, name='status_do_caso')
#Submissao_XGB = pd.concat([id_test_guardar, series_predicoes2], axis=1)

In [48]:
#Submissao_XGB.to_csv("usando_pos_weight2.csv", index=False)
